In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from pathlib import Path
from keras.preprocessing import image
import numpy as np
import tensorflow as tf

In [ ]:
train_dir = '/content/Indian-Currency-Classification-main/train'   

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
          #train_dir, 
    directory=r"C:\Users\ravip\Indian-Currency-Classification-main\Indian-Currency-Classification-main\train",
          labels='inferred', 
          label_mode='int',
          class_names=['10', '20', '50', '100', '200', '500', '2000'], 
          color_mode='rgb', 
          batch_size=32, 
          image_size=(128,128), 
          shuffle=True, 
          seed=123, 
          validation_split=0.2, 
          subset='training',
          interpolation='bicubic', 
          follow_links=False, 
          smart_resize=True
        )

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
          directory=r"C:\Users\ravip\Indian-Currency-Classification-main\Indian-Currency-Classification-main\train", 
          labels='inferred', 
          label_mode='int',
          class_names=['10', '20', '50', '100', '200', '500', '2000'], 
          color_mode='rgb', 
          batch_size=32, 
          image_size=(128,128), 
          shuffle=False, 
          seed=123, 
          validation_split=0.2, 
          subset='validation',
          interpolation='bicubic', 
          follow_links=False, 
          smart_resize=True
        )

In [ ]:
def normalize(image,label):
    """
    Returns normalized image and its label
    """ 
    image = tf.cast(image/255. ,tf.float32)
    return image,label

In [ ]:
train_dataset = train_dataset.map(normalize)
validation_dataset = validation_dataset.map(normalize)

In [ ]:
model = tf.keras.Sequential([
            
            tf.keras.layers.Conv2D(64, (1, 1), input_shape=(128, 128, 3)),
            tf.keras.layers.BatchNormalization(axis=-1),
            tf.keras.layers.Activation('relu'),
            
            tf.keras.layers.Conv2D(32, (3, 3)),
            tf.keras.layers.BatchNormalization(axis=-1),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

            tf.keras.layers.Conv2D(16, (3, 3)),
            tf.keras.layers.BatchNormalization(axis=-1),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.50),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dropout(0.30),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(7),
            tf.keras.layers.Activation('softmax')
          ])

In [ ]:
layer_names = []
for layer in model.layers[:8]:
    layer_names.append(layer.name)
print(layer_names)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
    metrics=['accuracy']
    )

In [ ]:
model1=model.fit(train_dataset,validation_data=validation_dataset,epochs=30)

In [ ]:
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
# Pre-processing the image
img = tf.keras.utils.load_img(r"C:\Users\ravip\Indian-Currency-Classification-main/Indian-Currency-Classification-main/train/100/new_100.jpg", target_size = (128,128))
img_tensor = tf.keras.utils.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis = 0)
img_tensor = img_tensor / 255.
  
# Print image tensor shape
print(img_tensor.shape)
  
# Print image
import matplotlib.pyplot as plt
plt.imshow(img_tensor[0])
plt.show()

In [ ]:
from keras import models
from keras import layers
# Outputs of the 8 layers, which include conv2D and max pooling layers
layer_outputs = [layer.output for layer in model.layers[:8]]
activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)
activations = activation_model.predict(img_tensor)

# Getting Activations of first layer
first_layer_activation = activations[1]

# shape of first layer activation
print(first_layer_activation.shape)

# 6th channel of the image after first layer of convolution is applied
plt.matshow(first_layer_activation[0, :, :, 6], cmap ='viridis')

# 15th channel of the image after first layer of convolution is applied
plt.matshow(first_layer_activation[0, :, :, 15], cmap ='viridis')


In [ ]:
import os
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
train_dir=r"C:\Users\ravip\Indian-Currency-Classification-main/Indian-Currency-Classification-main/train/10"
for i  in os.listdir(train_dir):
    img=tf.keras.utils.load_img(train_dir+'//'+i)
    plt.imshow(img)
    plt.show()

In [ ]:
val_loss, val_acc = model.evaluate(validation_dataset, verbose='auto')
print(f"Validation:\n\tloss:{val_loss} \n\taccuracy:{val_acc}")

In [ ]:
test_dir = r"C:\Users\ravip\Indian-Currency-Classification-main\Indian-Currency-Classification-main\test"

test_imageID = []
# r=root, d=directories, f = files

for r, d, f in os.walk(test_dir, topdown=False):
    for file in f:
      if '.jpg' in file:
          test_imageID.append(Path(file).stem)
test_imageID = sorted(test_imageID)

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
          test_dir, 
          labels='inferred', 
          label_mode=None,
          class_names = None,
          color_mode='rgb', 
          batch_size=1,
          shuffle = False,
          image_size=(128, 128), 
          interpolation='bicubic', 
          smart_resize=True
        )

In [ ]:
def normalize_test(image):
  """
    Returns normalized image and its label
  """
  image = tf.cast(image/255. ,tf.float32)
  return image

In [ ]:
# # Normalizing dataset
test_dataset = test_dataset.map(normalize_test)

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
pred_category = np.argmax(predictions,axis = 1)     

In [ ]:
labels_name = ['10', '20', '50', '100', '200', '500', '2000']
pred_output_labels = [labels_name[i] for i in pred_category]

In [ ]:
test_images = list(test_dataset.as_numpy_iterator()) 
num_test_images = len(test_images)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
acc = model1.history['accuracy']
val_acc = model1.history['val_accuracy']
loss = model1.history['loss']
val_loss = model1.history['val_loss']
epochs = range(1, len(loss) + 1)
#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
subplot_rows = num_test_images//6 + (1 if num_test_images%6!=0 else 0)
subplot_columns = num_test_images if num_test_images<6 else 6

plt.figure(figsize=(20, 20))
i = 0     

for images in test_images:
  ax = plt.subplot(subplot_rows, subplot_columns, i + 1)
  plt.imshow((np.squeeze(images) * 255).astype(np.uint8), cmap = 'gray')
  plt.title(pred_output_labels[i])
  plt.axis("off")
  i = i + 1

plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
image_path = "test/all-new-Rs-10-note-1.jpg"
new_img = image.load_img(image_path, target_size=(128, 128))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
img = img/255
print("Following is our prediction:")
prediction = model.predict(img)
d = prediction.flatten()
j = d.max()
for index,item in enumerate(d):
    if item == j:
        class_name = labels_name[index]
#ploting image with predicted class name 
plt.figure(figsize = (4,4))
plt.imshow(new_img)
plt.axis('off')
plt.title(class_name)
plt.show()

In [ ]:
model.save(r"C:\Users\ravip\Indian-Currency-Classification-main\Indian-Currency-Classification-main\my_model")  # Save the model 

In [ ]:
infer_model = tf.keras.models.load_model(r"C:\Users\ravip\Indian-Currency-Classification-main\Indian-Currency-Classification-main\my_model")


infer_model.summary()

In [ ]:
val_loss, val_acc = infer_model.evaluate(validation_dataset, verbose='auto')
print(f"Restored model Validation:\n\tloss:{val_loss} \n\taccuracy:{val_acc}")